In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/content/2012-2013-data-with-predictions-4-final.csv')


ParserError: Error tokenizing data. C error: EOF inside string starting at row 2022458

In [ ]:
df = df.sample(frac=0.1, random_state=42)

NameError: name 'df' is not defined

In [ ]:
df

NameError: name 'df' is not defined

In [ ]:
print(df.isnull().sum())

In [ ]:
irrelevant_columns = [
    'problem_log_id', 'problem_id', 'user_id', 'teacher_id',
    'school_id', 'answer_id', 'answer_text', 'actions', 'tutor_mode', 'skill'
]

# Drop them from the dataset
data = df.drop(columns=irrelevant_columns)

In [ ]:
print(data.isnull().sum())

In [ ]:
data = data.dropna()

In [ ]:
print(data.isnull().sum())

In [ ]:
data

In [ ]:
data['start_time'] = data['start_time'].str.split('.').str[0]
data['end_time'] = data['end_time'].str.split('.').str[0]

# Then, convert to datetime
data['start_time'] = pd.to_datetime(data['start_time'], format="%Y-%m-%d %H:%M:%S")
data['end_time'] = pd.to_datetime(data['end_time'], format="%Y-%m-%d %H:%M:%S")

# Calculate 'time_taken' in seconds
data['time_taken'] = (data['end_time'] - data['start_time']).dt.total_seconds()

# Convert to float
data['time_taken'] = data['time_taken'].astype(float)

# Drop the original datetime columns
data = data.drop(columns=['start_time', 'end_time'])

# Check the result
print(data[['time_taken']].head())

from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_columns = ['skill_id', 'problem_type', 'first_action', 'type']

# Apply Label Encoding
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

print(data.dtypes)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming your dataset is `data` and your target column is 'correct'
X = data.drop(columns=['correct'])  # Features
y = data['correct']  # Target
y = y.astype(int)

# Stratified sampling to ensure equal class distribution
X_sampled, X_test, y_sampled, y_test = train_test_split(X, y, test_size=0.99, stratify=y, random_state=42)

# Train a Random Forest model to rank features
model = RandomForestClassifier(random_state=42)
model.fit(X_sampled, y_sampled)

# Plot feature importance
importance = pd.Series(model.feature_importances_, index=X.columns)
importance.sort_values(ascending=False).plot(kind='bar', figsize=(10, 6), title='Feature Importance')
plt.show()


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming y_test and y_pred are already defined
# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
print(f"F1 Score: {f1:.2f}")

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Optionally, visualize the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=set(y_test))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Assuming your dataset is `data` and your target column is 'correct'
X = data[['attempt_count', 'hint_count', 'bottom_hint', 'time_taken', 'first_action', 'ms_first_response', 'overlap_time',
          'Average_confidence(CONCENTRATING)', 'Average_confidence(BORED)']] # Features
y = data['correct']  # Target
y = y.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, stratify=y, random_state=42)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Conv1D, MaxPooling1D, Embedding, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import to_categorical
num_classes = y.nunique()

# Convert labels to categorical for neural network use
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Define models and hyperparameter grids for grid search
models = {
    'Linear Regression': LinearRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

param_grids = {
    'Linear Regression': {},  # No hyperparameters for basic linear regression
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance']
    },
    'Naive Bayes': {},  # No hyperparameters typically tuned
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30]
    },
    'Decision Tree': {
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10]
    }
}

# Placeholder for results
results = []

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    if param_grids[model_name]:  # If hyperparameters are defined
        grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name], scoring='f1_weighted', cv=5)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
    else:
        best_model = model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_model.predict(X_test)
    if y_pred.dtype != int and model_name != 'Linear Regression':
      y_pred = (y_pred > 0.5).astype(int)
    if model_name != 'Linear Regression':  # Skip non-classification metrics for linear regression
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')

        results.append({'Model': model_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1})

# Neural Network models
def create_rnn_model(input_dim, num_classes):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128),
        SimpleRNN(64),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_lstm_model(input_dim, num_classes):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128),
        LSTM(64),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_capsule_network(input_dim, num_classes):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=128),
        Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'),
        GlobalAveragePooling1D(),
        Dense(num_classes, activation='softmax')
    ])
    return model

neural_networks = {
    'RNN': create_rnn_model(input_dim=10000, num_classes=num_classes),
    'LSTM': create_lstm_model(input_dim=10000, num_classes=num_classes),
    'Capsule Network': create_capsule_network(input_dim=10000, num_classes=num_classes)
}

for name, model in neural_networks.items():
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
    model.fit(X_train, y_train_cat, epochs=10, batch_size=64)
    metrics = model.evaluate(X_test, y_test_cat)
    results.append({
        'Model': name,
        'Accuracy': metrics[1],
        'Precision': metrics[2],
        'Recall': metrics[3],
        'F1 Score': 2 * (metrics[2] * metrics[3]) / (metrics[2] + metrics[3])  # Harmonic mean of precision and recall
    })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Display results
print(results_df.sort_values(by='F1 Score', ascending=False))

# Plot F1 scores
plt.figure(figsize=(12, 6))
plt.bar(results_df['Model'], results_df['F1 Score'], color='skyblue', edgecolor='black')
plt.xlabel('Model', fontsize=12)
plt.ylabel('F1 Score', fontsize=12)
plt.title('F1 Score Comparison of Models', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

# Display the results DataFrame
print(results_df)

In [ ]:
print(df.columns)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Step 1: Identify weak skills for each student
incorrect_answers = df[df['correct'] == 0]

# Group by `user_id` and aggregate their weak `skill_id`
student_weak_skills = incorrect_answers.groupby('user_id')['skill_id'].apply(
    lambda x: ', '.join(map(str, x.dropna().unique()))
).reset_index()
student_weak_skills.rename(columns={'skill_id': 'weak_skills'}, inplace=True)

# Step 2: Prepare the resources (problems with skill associations)
resource_skills = df[['problem_id', 'skill_id']].drop_duplicates()
resource_skills['skill_id'] = resource_skills['skill_id'].astype(str)

# Step 3: Combine skills data for TF-IDF vectorization
combined_skills = pd.concat([
    student_weak_skills['weak_skills'],
    resource_skills['skill_id']
], axis=0).fillna('')

# Vectorize skill data using TF-IDF
vectorizer = TfidfVectorizer()
skills_tfidf = vectorizer.fit_transform(combined_skills)

# Step 4: Compute similarity
# Split back into student and resource vectors
student_skills_tfidf = skills_tfidf[:len(student_weak_skills)]
resource_skills_tfidf = skills_tfidf[len(student_weak_skills):]

similarity_matrix = cosine_similarity(student_skills_tfidf, resource_skills_tfidf)

# Step 5: Create a recommendation function
def recommend_resources(student_id, top_n=5):
    # Check if the student exists in the weak_skills data
    if student_id not in student_weak_skills['user_id'].values:
        print(f"Student ID {student_id} not found.")
        return pd.DataFrame()

    # Get the student's index in the similarity matrix
    student_index = student_weak_skills[student_weak_skills['user_id'] == student_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[student_index]))

    # Rank resources by similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_resource_indices = [i[0] for i in similarity_scores[:top_n]]
    recommended_resources = resource_skills.iloc[top_resource_indices]

    return recommended_resources

# Step 6: Test the recommender system
student_id = 52535  # Replace with an actual `user_id` from the dataset
recommendations = recommend_resources(student_id, top_n=5)

# Map skill IDs to skill names
skill_mapping = {85.0: "Algebra"}  # Ensure this dictionary includes all relevant skill IDs

# Avoid SettingWithCopyWarning by making a copy
recommendations = recommendations.copy()

# Ensure matching data types
recommendations.loc[:, 'skill_id'] = recommendations['skill_id'].astype(float)

# Map skill IDs and handle missing mappings
recommendations.loc[:, 'skill_name'] = recommendations['skill_id'].map(skill_mapping).fillna("Unknown Skill")

# Display recommendations with skill names
print(recommendations[['problem_id', 'skill_name']])

# Visualization: Plot recommendations
plt.figure(figsize=(10, 6))
plt.bar(recommendations['problem_id'].astype(str), recommendations['skill_name'])
plt.xlabel('Problem ID', fontsize=12)
plt.ylabel('Skill Name', fontsize=12)
plt.title(f'Recommended Problems for Student {student_id}', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()